In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Installation

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install transformers==4.51.3
    !pip install --no-deps unsloth

### Unsloth and Model

In [ ]:
import json
from datasets import Dataset
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Can increase for longer reasoning traces
lora_rank = 16 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "ministral/Ministral-3b-instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = False,
    max_lora_rank = lora_rank,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.2: Fast Mistral patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/2.96G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/2.97G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/510 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

ministral/Ministral-3b-instruct does not have a padding token! Will use pad_token = <unk>.


### LoRA config

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

Unsloth 2025.6.2 patched 14 layers with 14 QKV layers, 14 O layers and 14 MLP layers.


### Data Prep

In [ ]:
alpaca_prompt = """Chuyển đổi câu hỏi tiếng Việt thành câu lệnh SQL dựa trên schema được cung cấp.

### Context:
{}

### Question:
{}

### Answer:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    contexts = examples["context"]
    questions       = examples["question"]
    answers      = examples["answer"]
    texts = []
    for context, question, answer in zip(contexts, questions, answers):
        text = alpaca_prompt.format(context, question, answer) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

file_path = "train.json"
# 1. Tải dữ liệu từ data.json
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Chuyển đổi sang Hugging Face Dataset (không cần chia train/test cho ví dụ này)
dataset = Dataset.from_list(data)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/9693 [00:00<?, ? examples/s]

### Train the model

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1,
        #max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/9693 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 9,693 | Num Epochs = 1 | Total steps = 1,211
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 18,350,080/3,334,066,176 (0.55% trained)


Step,Training Loss
1,2.514300
2,2.765300
3,1.996900
4,2.202100
5,1.805100
6,2.164700
7,1.785500
8,1.892900
9,1.622300
10,1.823500


### Save model

In [ ]:
model.save_pretrained("/content/drive/MyDrive/Ministral-3b-v2")  # Local saving
tokenizer.save_pretrained("/content/drive/MyDrive/Ministral-3b-v2")

('/content/drive/MyDrive/Ministral-3b-v2/tokenizer_config.json',
 '/content/drive/MyDrive/Ministral-3b-v2/special_tokens_map.json',
 '/content/drive/MyDrive/Ministral-3b-v2/tokenizer.json')

### Inference

In [ ]:
dtype = None
load_in_4bit = False

if True:
    from unsloth import FastLanguageModel

    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="/content/drive/MyDrive/Ministral-3b",
        max_seq_length=max_seq_length,
        dtype=dtype,
        load_in_4bit=load_in_4bit,
    )
    FastLanguageModel.for_inference(model)

alpaca_prompt = """Chuyển đổi câu hỏi tiếng Việt sang câu lệnh SQL dựa trên schema sau:

### Context:
{}

### Question:
{}

### Answer:
{}"""
FastLanguageModel.for_inference(model)
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "CREATE TABLE hoc_sinh (tuoi INTEGER, hoc_sinh_id VARCHAR); CREATE TABLE has_pet (tuoi INTEGER, hoc_sinh_id VARCHAR)",  # context
            "Tìm độ tuổi trung bình của những học sinh không nuôi thú cưng.",  # question
            "",  # answer - leave this blank for generation!
        )
    ],
    return_tensors="pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=64)

==((====))==  Unsloth 2025.6.1: Fast Mistral patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

<s> Chuyển đổi câu hỏi tiếng Việt sang câu lệnh SQL dựa trên schema sau:

### Context:
CREATE TABLE hoc_sinh (tuoi INTEGER, hoc_sinh_id VARCHAR); CREATE TABLE has_pet (tuoi INTEGER, hoc_sinh_id VARCHAR)

### Question:
Tìm độ tuổi trung bình của những học sinh không nuôi thú cưng.

### Answer:
select avg ( tuoi ) from hoc_sinh where id not in ( select t1.id from may_bay as t1 join may_bay as t2 on t1.id may_bay = t2.id may_bay where t2.ten may_
